In [1]:
# Run this again after editing submodules so Colab uses the updated versions
from citylearn import  CityLearn
from pathlib import Path
from agent_final import Agent
import numpy as np                                                                                                                                                                                      
import torch
import matplotlib.pyplot as plt

In [2]:
# Load environment
climate_zone = 5
end_time = 8759 # total number of hours to train for
cost_analysis = 1 # analyze costs every x hours
checkpoint = 24 * 7 # save parameters every x hours

params = {'data_path':Path("data/Climate_Zone_"+str(climate_zone)), 
        'building_attributes':'building_attributes.json', 
        'weather_file':'weather_data.csv', 
        'solar_profile':'solar_generation_1kW.csv', 
        'carbon_intensity':'carbon_intensity.csv',
        'building_ids':["Building_"+str(i) for i in [1]],
        'buildings_states_actions':'buildings_state_action_space.json', 
        'simulation_period': (0, end_time), 
        'cost_function': ['ramping','1-load_factor','average_daily_peak','peak_demand','net_electricity_consumption','carbon_emissions'], 
        'central_agent': False,
        'cost_analysis' : cost_analysis,
        'save_memory': False }

# Contain the lower and upper bounds of the states and actions, to be provided to the agent to normalize the variables between 0 and 1.
env = CityLearn(**params)
env.pretrain_baseline_model((0, end_time))

observations_spaces, actions_spaces = env.get_state_action_spaces()

# Provides information on Building type, Climate Zone, Annual DHW demand, Annual Cooling Demand, Annual Electricity Demand, Solar Capacity, and correllations among buildings
building_info = env.get_building_information()

Time taken to add RBC pretrained model (sec): 1.299


In [3]:
params_agent = {'building_ids':["Building_"+str(i) for i in [1]],
                 'buildings_states_actions':'buildings_state_action_space.json', 
                 'building_info':building_info,
                 'observation_spaces':observations_spaces, 
                 'action_spaces':actions_spaces}

# Instantiating the control agent(s)
agents = Agent(**params_agent)

state = env.reset()
rewards = []
done = False

action, coordination_vars = agents.select_action(state)    
while not done:
    next_state, reward, done, _ = env.step(action)
    action_next, coordination_vars_next = agents.select_action(next_state)
    agents.add_to_buffer(state, action, reward, next_state, done, coordination_vars, coordination_vars_next)
    coordination_vars = coordination_vars_next
    state = next_state
    action = action_next
    
    rewards.append(reward)
    
    try:
        print(f"\r {env.time_step}\t{env.costs_periodic[-1]['total']}", end="")
    except:
        print(f"\r {env.time_step}", end="")

 8759	0.9122704117613826

In [4]:
env.cost((0, end_time))

({'ramping': 0.860724,
  '1-load_factor': 0.9744107516059408,
  'average_daily_peak': 0.8213672,
  'peak_demand': 0.8966831,
  'net_electricity_consumption': 0.9597347,
  'carbon_emissions': 0.960693,
  'total': 0.9122687841695072,
  'coordination_score': 0.8882962562503698},
 <citylearn.CityLearn at 0x7fc642306100>)

In [5]:
state = env.reset()
for e in range(end_time): 
    action, coordination_vars = agents.select_action(state)    
    next_state, reward, done, _ = env.step(action)
    state = next_state
    action = action_next

In [8]:
import pandas as pd

In [9]:
building_number = 'Building_1'
d = env.buildings[building_number].__dict__#'time_step
list_param1 = ['electric_consumption_cooling', 
       'electric_consumption_cooling_storage', 'electric_consumption_dhw', 
       'electric_consumption_dhw_storage','cooling_device_to_building',
       'cooling_storage_to_building', 'cooling_device_to_storage', 'cooling_storage_soc', 
       'dhw_heating_device_to_building', 'dhw_storage_to_building', 'dhw_heating_device_to_storage', 
       'dhw_storage_soc', 'electrical_storage_electric_consumption', 'electrical_storage_soc']
d1= {}
d1['net_electric_consumption'] = env.net_electric_consumption
for i in list_param1:
    d1[i] = d[i]
df1 = pd.DataFrame.from_dict(d1)
df = pd.DataFrame.from_dict(d['sim_results'])
df1.to_csv('milp/results_RL.csv') 
df.to_csv('milp/sim_results_RL.csv')

In [12]:
env.cost_detailed_view()['Building_1']['cost']

{'ramping': 0.9969139174448106,
 'c_score': [0.9969139174448106,
  0.9931400715045118,
  0.9853549517221527,
  0.9763892274131761],
 '1-load_factor': 0.9931400715045118,
 'average_daily_peak': 0.9853549517221527,
 'peak_demand': 0.9763892274131761,
 'net_electricity_consumption': 0.9986571915102009,
 'carbon_emissions': 0.960693,
 'total': 0.9851913935569973,
 'coordination_score': 0.9879495420211628}